---

**設定：Setting**


In [ ]:
# 建物レベル最適運転計画（簡潔版）
# - 入力: ac_control_processed_Clea.csv, power_meter_processed_Clea.csv
# - 出力: 予測モデル（MultiOutput）で室内温度/電力を予測し、天気予報に基づき簡易に最適計画を算出
import os
import sys
from pathlib import Path

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.multioutput import MultiOutputRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import train_test_split
from sklearn.metrics import r2_score, mean_absolute_error

# 建物レベル横持ち特徴量の作成 + 単一モデル(XGBoost)学習（多出力）
from xgboost import XGBRegressor
from sklearn.multioutput import MultiOutputRegressor

# Safe JP font setup without warnings
import matplotlib as mpl
import matplotlib.pyplot as plt
from matplotlib import font_manager
import warnings

# 使える候補（環境にあるものを自動選択）
preferred = [
    "Hiragino Sans",  # macOS
    "Hiragino Kaku Gothic ProN",
    "Yu Gothic",  # Windows
    "MS Gothic",
    "AppleGothic",  # macOS 代替
    "Arial Unicode MS",  # 広範囲
    "DejaVu Sans",  # 最終手段（英数・一部日本語）
]

# システムフォント一覧から存在チェック
available = set(font_manager.get_font_names())
font_name = next((f for f in preferred if f in available), "DejaVu Sans")

# 参照しない（存在しないフォント名をrcParamsから除外）
mpl.rcParams["font.family"] = font_name
mpl.rcParams["font.sans-serif"] = [font_name]
mpl.rcParams["axes.unicode_minus"] = False

# 旧セルで指定した未インストール名をクリア（必要なら）
for k in ["font.family", "font.sans-serif"]:
    mpl.rcParams[k] = [font_name]

# findfont の警告を抑制
mpl.set_loglevel("error")
warnings.filterwarnings("ignore", message=r"findfont: Font family '.*' not found")

print(f"Using font: {font_name}")


# プロジェクトルート
cwd = Path.cwd()
project_root = cwd.parent if cwd.name == "notebooks" else cwd
if str(project_root) not in sys.path:
    sys.path.append(str(project_root))

print("Project root:", project_root)

Using font: Hiragino Sans
Project root: /Users/toukouken/Documents/MENTERU/AIrux8_opti_logic


In [ ]:
# パス設定（必要に応じて変更）
DATA_DIR = project_root / "data/02_PreprocessedData/Clea"
AC_CONTROL_CSV = DATA_DIR / "ac_control_processed_Clea.csv"
WEATHER_CSV = DATA_DIR / "weather_processed_Clea.csv"
POWER_METER_CSV = DATA_DIR / "power_meter_processed_Clea.csv"

# 天気予報（本来はAPI取得、ここでは簡潔のため既存の予報CSVを指定可能）
FORECAST_CSV = (
    project_root / "data/04_PlanningData/Clea/weather_forecast_20250929_20251004.csv"
)

print("AC control:", AC_CONTROL_CSV)
print("Power:", POWER_METER_CSV)
print("Forecast:", FORECAST_CSV)

AC control: /Users/toukouken/Documents/MENTERU/AIrux8_opti_logic/data/02_PreprocessedData/Clea/ac_control_processed_Clea.csv
Power: /Users/toukouken/Documents/MENTERU/AIrux8_opti_logic/data/02_PreprocessedData/Clea/power_meter_processed_Clea.csv
Forecast: /Users/toukouken/Documents/MENTERU/AIrux8_opti_logic/data/04_PlanningData/Clea/weather_forecast_20250929_20251004.csv


---

**前処理：pre-Processing**


In [ ]:
# 計測ユーティリティ
import time, cProfile, pstats, io


def section_time(label, func, *args, **kwargs):
    t0 = time.perf_counter()
    out = func(*args, **kwargs)
    dt = time.perf_counter() - t0
    print(f"[TIME] {label}: {dt:.2f}s")
    return out


def profile_block(run_callable, sort_by="cumtime", lines=30):
    pr = cProfile.Profile()
    pr.enable()
    run_callable()
    pr.disable()
    s = io.StringIO()
    pstats.Stats(pr, stream=s).sort_stats(sort_by).print_stats(lines)
    print(s.getvalue())

In [ ]:
# データ読み込みと前処理（簡潔版）
assert AC_CONTROL_CSV.exists(), f"not found: {AC_CONTROL_CSV}"
assert POWER_METER_CSV.exists(), f"not found: {POWER_METER_CSV}"

ac = pd.read_csv(AC_CONTROL_CSV)
pw = pd.read_csv(POWER_METER_CSV)

# 列名BOM除去と datetime 統一
ac.columns = ac.columns.str.replace("\ufeff", "")
pw.columns = pw.columns.str.replace("\ufeff", "")

datetime_col = "Datetime" if "Datetime" in ac.columns else "datetime"
ac[datetime_col] = pd.to_datetime(ac[datetime_col])
pw[datetime_col] = pd.to_datetime(pw[datetime_col])

# 時刻列を統一
ac["Datetime"] = ac["Datetime"] if "Datetime" in ac.columns else ac["datetime"]
ac["Datetime"] = pd.to_datetime(ac["Datetime"])

# 室内機の識別（unit_id or indoor_id 的な列名推定）
UNIT_ID = "A/C Name"
# 固定順のユニット一覧
units_fixed = sorted(ac[UNIT_ID].dropna().unique().tolist())
units_fixed

['A-25',
 'A-26',
 'D-1南1',
 'D-2北1',
 'D-3南2',
 'D-4北2',
 'D-5南1',
 'D-6北1',
 'D-7南2',
 'D-8北2',
 'E-10南2',
 'E-11南3',
 'E-12南4',
 'E-13北1',
 'E-14北2',
 'E-15北3',
 'E-16北4',
 'E-17',
 'E-9南1',
 'F-18',
 'F-19',
 'F-20',
 'G-21',
 'G-22',
 'G-23',
 'G-24']

In [ ]:
# 必要そうな列の推定
col_map = {
    "set_temp": None,
    "mode": None,
    "fan": None,
    "onoff": None,
    "indoor_temp": None,
    "outdoor_temp": None,
    "solar": None,
}
for c in ac.columns:
    cl = c.lower()
    if col_map["set_temp"] is None and ("set" in cl and "temp" in cl):
        col_map["set_temp"] = c
    if col_map["mode"] is None and "mode" in cl:
        col_map["mode"] = c
    if col_map["fan"] is None and "fan" in cl:
        col_map["fan"] = c
    if col_map["onoff"] is None and (
        "on/off" in cl or "on_off" in cl or "onoff" in cl or "a/c on/off" in cl
    ):
        col_map["onoff"] = c
    if col_map["indoor_temp"] is None and ("indoor" in cl and "temp" in cl):
        col_map["indoor_temp"] = c
    if col_map["outdoor_temp"] is None and ("outdoor" in cl and "temp" in cl):
        col_map["outdoor_temp"] = c
    if col_map["solar"] is None and ("solar" in cl):
        col_map["solar"] = c

In [ ]:
from datetime import datetime

import pandas as pd
from prettytable import PrettyTable


class DataManager:
    @classmethod
    def show_status(self, df: pd.DataFrame):
        table = PrettyTable(
            ["Variable", "Type", "Missing Values", "Duplicates", "Outliers"]
        )

        for col in df.columns:
            # データ型の判定
            if pd.api.types.is_numeric_dtype(df[col]):
                col_type = "Numerical"
            else:
                col_type = "Categorical"

            # 欠損値
            missing_values = df[col].isnull().sum()

            # 重複数
            duplicates = df.duplicated(subset=[col]).sum()

            # 外れ値（数値型のみ）
            if col_type == "Numerical":
                mean = df[col].mean()
                std = df[col].std()
                outliers = ((df[col] - mean).abs() > 3 * std).sum()
            else:
                outliers = "N/A"

            table.add_row([col, col_type, missing_values, duplicates, outliers])
        print(table)

    @classmethod
    def extract_term_data(
        cls, target_col: str, df: pd.DataFrame, start_time: datetime, end_time: datetime
    ):
        df[target_col] = pd.to_datetime(df[target_col])
        filtered_df = df[(df[target_col] >= start_time) & (df[target_col] <= end_time)]
        return filtered_df

    @classmethod
    def check_date_sequence(cls, df: pd.DataFrame, date_col: str = "date"):
        dates = pd.to_datetime(df[date_col]).sort_values().reset_index(drop=True)
        expected = pd.date_range(start=dates.min(), end=dates.max(), freq="D")
        missing = expected.difference(dates)
        return {
            "is_continuous": len(missing) == 0,
            "missing_dates": missing.strftime("%Y-%m-%d").tolist(),
        }

In [23]:
DataManager.show_status(ac)
print(len(ac))

+---------------------+-------------+----------------+------------+----------+
|       Variable      |     Type    | Missing Values | Duplicates | Outliers |
+---------------------+-------------+----------------+------------+----------+
|       Datetime      | Categorical |       0        |  3205254   |   N/A    |
|         Date        | Categorical |       0        |  3335400   |   N/A    |
|       A/C Name      | Categorical |       0        |  3335830   |   N/A    |
|    Outdoor Temp.    |  Numerical  |       0        |  3335811   |    0     |
|     Indoor Temp.    |  Numerical  |       0        |  3335810   |  13070   |
| A/C Set Temperature |  Numerical  |       0        |  3335838   |  13510   |
|      A/C ON/OFF     |  Numerical  |       0        |  3335854   |    0     |
|       A/C Mode      |  Numerical  |     44361      |  3335852   |    0     |
|    A/C Fan Speed    |  Numerical  |    2352277     |  3335850   |   1373   |
|  Naive Energy Level |  Numerical  |       0       

ここで、カテゴリカルを numerical へ変換する処理を加える


In [22]:
activate_replace_dict = {"OFF": 0, "ON": 1}
mode_replace_dict = {"FAN": 0, "COOL": 1, "HEAT": 2}
fan_replace_dict = {"Low": 0, "Medium": 1, "High": 2, "Top": 3, "Auto": 4}
ac["A/C ON/OFF"] = (
    ac["A/C ON/OFF"].astype(str).str.strip().str.upper().map(activate_replace_dict)
)
ac["A/C Mode"] = (
    ac["A/C Mode"].astype(str).str.strip().str.upper().map(mode_replace_dict)
)
ac["A/C Fan Speed"] = (
    ac["A/C Fan Speed"].astype(str).str.strip().str.title().map(fan_replace_dict)
)

In [24]:
DataManager.show_status(pw)
print(len(pw))

+------------+-------------+----------------+------------+----------+
|  Variable  |     Type    | Missing Values | Duplicates | Outliers |
+------------+-------------+----------------+------------+----------+
|  Datetime  | Categorical |       0        |  1869023   |   N/A    |
|    Date    | Categorical |       0        |  2517503   |   N/A    |
|  Mesh ID   |  Numerical  |       0        |  2517955   |    0     |
| PM Addr ID |  Numerical  |       0        |  2517950   |    0     |
|  Phase A   |  Numerical  |       0        |  2515287   |  67473   |
|  Phase B   |  Numerical  |       0        |  2517958   |    0     |
|  Phase C   |  Numerical  |       0        |  2517958   |    0     |
| Total_kWh  |  Numerical  |       0        |  2515287   |  67473   |
+------------+-------------+----------------+------------+----------+
2517959


In [25]:
# 室内機(ac)・室外機(pw)をグルーピングして、時刻別（時刻×ユニット）の横持ちテーブルに変換

import pandas as pd
import numpy as np


# 1) 時刻列を統一＆時単位に丸め
def to_hourly(df, dt_col):
    df = df.copy()
    df[dt_col] = pd.to_datetime(df[dt_col])
    df["Datetime_hour"] = df[dt_col].dt.floor("H")
    return df


# 2) 室内機（A/C Name）× 時刻で集約 → 横持ち
#    - 集約指標の例:
#      set_temp: 平均, mode/fan/onoff: 最頻値, indoor_temp: 平均
def pivot_indoor(
    ac: pd.DataFrame,
    unit_col="A/C Name",
    dt_col="Datetime",
    set_temp_col=None,
    mode_col=None,
    fan_col=None,
    onoff_col=None,
    indoor_temp_col=None,
):
    ac1 = to_hourly(ac, dt_col)

    agg_map = {}
    if set_temp_col:
        agg_map[set_temp_col] = "mean"
    if indoor_temp_col:
        agg_map[indoor_temp_col] = "mean"

    # 最頻値集計（mode/fan/onoff）
    def mode_agg(s):
        s = pd.to_numeric(s, errors="coerce")
        s = s.dropna().astype(int)
        return s.mode().iloc[0] if len(s) else np.nan

    if mode_col:
        agg_map[mode_col] = mode_agg
    if fan_col:
        agg_map[fan_col] = mode_agg
    if onoff_col:
        agg_map[onoff_col] = mode_agg

    g = ac1.groupby(["Datetime_hour", unit_col], as_index=False).agg(agg_map)

    # wide化（列名: 指標__ユニット）
    wide_parts = []
    for col, fn in agg_map.items():
        p = g.pivot(index="Datetime_hour", columns=unit_col, values=col)
        p.columns = [f"{col}__{c}" for c in p.columns]
        wide_parts.append(p)
    ac_wide = pd.concat(wide_parts, axis=1).sort_index()
    return ac_wide  # index=時刻(時), columns= 指標__A/C Name


# 3) 室外機（Mesh ID + PM Addr ID）× 時刻で集約 → 横持ち
#    - Phaseは無視、Total_kWhを合計
def pivot_outdoor(
    pw: pd.DataFrame,
    dt_col="Datetime",
    mesh_col="Mesh ID",
    addr_col="PM Addr ID",
    kwh_col="Total_kWh",
):
    pw1 = to_hourly(pw, dt_col).copy()
    # 室外機ID生成
    pw1["ODU_ID"] = pw1[mesh_col].astype(str) + "-" + pw1[addr_col].astype(str)
    # 時刻×ODUで合計
    kwh_actual_col = (
        kwh_col
        if kwh_col in pw1.columns
        else ("Total_KWh" if "Total_KWh" in pw1.columns else None)
    )
    if kwh_actual_col is None:
        raise ValueError("Total_kWh (or Total_KWh) 列が見つかりません")
    g = pw1.groupby(["Datetime_hour", "ODU_ID"], as_index=False)[kwh_actual_col].sum()
    pw_wide = g.pivot(index="Datetime_hour", columns="ODU_ID", values=kwh_actual_col)
    pw_wide.columns = [f"total_kwh__{c}" for c in pw_wide.columns]
    pw_wide = pw_wide.sort_index()
    return pw_wide  # index=時刻(時), columns= total_kwh__ODU_ID


# 4) 使い方例
# 室内機側の列名をプロジェクトに合わせて指定してください
ac_wide = pivot_indoor(
    ac,
    unit_col="A/C Name",
    dt_col="Datetime" if "Datetime" in ac.columns else "datetime",
    set_temp_col=col_map.get("set_temp"),
    mode_col=col_map.get("mode"),
    fan_col=col_map.get("fan"),
    onoff_col=col_map.get("onoff"),
    indoor_temp_col=col_map.get("indoor_temp"),
)

# 室外機側
pw_wide = pivot_outdoor(
    pw,
    dt_col="Datetime" if "Datetime" in pw.columns else "datetime",
    mesh_col="Mesh ID",
    addr_col="PM Addr ID",
    kwh_col="Total_kWh",  # or 'Total_KWh'
)

# 5) 時刻キーで突き合わせ（必要なら）
hourly_merged = ac_wide.join(pw_wide, how="outer").sort_index()
print(ac_wide.shape, pw_wide.shape, hourly_merged.shape)

/var/folders/8l/p7_mk6qs1y9dsdvmzt6qd68w0000gn/T/ipykernel_93752/2799536464.py:11: FutureWarning: 'H' is deprecated and will be removed in a future version, please use 'h' instead.
  df["Datetime_hour"] = df[dt_col].dt.floor("H")
/var/folders/8l/p7_mk6qs1y9dsdvmzt6qd68w0000gn/T/ipykernel_93752/2799536464.py:11: FutureWarning: 'H' is deprecated and will be removed in a future version, please use 'h' instead.
  df["Datetime_hour"] = df[dt_col].dt.floor("H")


(10894, 130) (10833, 21) (10896, 151)


---

**学習用に欠損を処理**


In [ ]:
DataManager.show_status(hourly_merged)
print(len(hourly_merged))

+------------------------------+-----------+----------------+------------+----------+
|           Variable           |    Type   | Missing Values | Duplicates | Outliers |
+------------------------------+-----------+----------------+------------+----------+
|  A/C Set Temperature__A-25   | Numerical |      245       |   10804    |    0     |
|  A/C Set Temperature__A-26   | Numerical |      245       |   10821    |   245    |
| A/C Set Temperature__D-1南1  | Numerical |      135       |   10808    |    10    |
| A/C Set Temperature__D-2北1  | Numerical |      134       |   10811    |   311    |
| A/C Set Temperature__D-3南2  | Numerical |      134       |   10808    |    50    |
| A/C Set Temperature__D-4北2  | Numerical |      134       |   10799    |   226    |
| A/C Set Temperature__D-5南1  | Numerical |      134       |   10805    |    0     |
| A/C Set Temperature__D-6北1  | Numerical |      134       |   10805    |    0     |
| A/C Set Temperature__D-7南2  | Numerical |      134       |

In [ ]:
# hourly_merged.to_csv("../data/base/hourly_merged.csv", index = True)

---

# 機械学習モデルの開発


## 欠損値補完

欠損値補完は、直近 1 週間の同じ時刻の値の平均値を代入<br>_但し、その中に Nan が含まれていれば、nan はカウントしない_


In [28]:
import pandas as pd

hourly_merged = pd.read_csv(
    "../data/base/hourly_merged.csv",
    parse_dates=["Datetime_hour"],  # ← ここを実際の列名に
    index_col="Datetime_hour",  # ← 同上
)
hourly_merged = hourly_merged.sort_index()

In [35]:
import pandas as pd
import numpy as np


def fill_missing_with_same_time_last_week(
    df: pd.DataFrame,
    *,
    num_cols: list[str] | None = None,
    cat_cols: list[str] | None = None,
    days: int = 7,
    numeric_fallback: float = 0.0,  # 履歴ゼロのとき数値を何で埋めるか（既定=0.0）
    categorical_fallback: object = 0.0,  # 履歴ゼロのときカテゴリを何で埋めるか（既定=None→NaNのまま）
    infer_categorical_max_unique: int = 20,  # 自動推定用：ユニーク数が小さい数値列はカテゴリ扱いに回す閾値
) -> pd.DataFrame:
    """
    DatetimeIndex の時系列に対して、同一“時刻”の直近N日で補完する。
      - 数値列: NaN無視の平均（履歴ゼロは numeric_fallback）
      - カテゴリ列: NaN無視の最頻値（履歴ゼロは categorical_fallback）
      - 未来情報は使わない（過去1..N日だけ）
    """
    if not isinstance(df.index, pd.DatetimeIndex):
        raise ValueError("df.index は DatetimeIndex である必要があります。")
    df = df.sort_index()

    # --- 列の自動推定 ---
    if num_cols is None:
        num_cols = df.select_dtypes(include=[np.number]).columns.tolist()
    if cat_cols is None:
        # 文字列/カテゴリ型はカテゴリ扱い
        cat_cols = set(
            df.select_dtypes(include=["object", "category", "bool"]).columns.tolist()
        )
        # 数値でも「ユニークが少ない」ものはカテゴリ扱いに回す（モード補完の対象）
        for c in num_cols:
            # ユニーク計算コストを抑えるために欠損除外で
            nunq = df[c].nunique(dropna=True)
            if nunq <= infer_categorical_max_unique:
                cat_cols.add(c)
        cat_cols = list(cat_cols)
        # カテゴリ扱いに回した数値列は数値リストから除外
        num_cols = [c for c in num_cols if c not in cat_cols]

    out = df.copy()

    # ===== 数値列：同時刻の過去1..N日の平均 =====
    if num_cols:
        derive_from_categoricals = [
            c for c in num_cols if any(p in c for p in ("Mode", "Fan Speed", "ON/OFF"))
        ]

        # “純粋な数値列”（上の以外）— 順序保持
        pure_numerical_cols = [c for c in num_cols if c not in derive_from_categoricals]
        shifted_num = [
            df[num_cols].shift(freq=pd.Timedelta(days=k)) for k in range(1, days + 1)
        ]
        # 合計と有効数（NaNを数えない）
        sum_vals = None
        count_vals = None
        for s in shifted_num:
            sum_vals = s if sum_vals is None else sum_vals.add(s, fill_value=0.0)
            c = s.notna().astype(int)
            count_vals = c if count_vals is None else count_vals.add(c, fill_value=0)
        mean_same_time = sum_vals.divide(count_vals).where(
            count_vals > 0, other=float(numeric_fallback)
        )
        mean_same_time_int = (
            (sum_vals / count_vals)
            .where(count_vals > 0, other=float(numeric_fallback))
            .round()
            .astype("Int64")  # 欠損対応の整数dtype
        )
        out[pure_numerical_cols] = out[pure_numerical_cols].fillna(mean_same_time)
        out[derive_from_categoricals] = out[derive_from_categoricals].fillna(
            mean_same_time_int
        )

    # ===== カテゴリ列：同時刻の過去1..N日の最頻値（mode） =====
    if cat_cols:
        shifted_cat = [
            df[cat_cols].shift(freq=pd.Timedelta(days=k)) for k in range(1, days + 1)
        ]
        # まとめて連結（列は MultiIndex: (k, col)）
        cat_concat = pd.concat(shifted_cat, axis=1, keys=range(1, days + 1))

        modes_df = pd.DataFrame(index=df.index, columns=cat_cols, dtype=object)
        for col in cat_cols:
            # この列の過去k日の同時刻だけを横持ち（列=各k）
            sub = cat_concat.xs(col, axis=1, level=1)  # shape: (T, days)
            # 行ごとモード（複数モード時は先頭を採用）
            # DataFrame.mode(axis=1) は複数列返す可能性があるので iloc[:,0] を使う
            m = sub.mode(axis=1, dropna=True)
            mode_series = (
                m.iloc[:, 0]
                if not m.empty
                else pd.Series(index=sub.index, dtype=object)
            )
            modes_df[col] = mode_series

        # 履歴ゼロ（= modeがNaN）のところだけ categorical_fallback で埋めたい場合
        if categorical_fallback is not None:
            modes_df = modes_df.fillna(categorical_fallback)

        # 元が欠損の所だけ置換（要素アライン）
        out[cat_cols] = out[cat_cols].where(~out[cat_cols].isna(), other=modes_df)

        # category dtype を壊さないよう、可能なら型を復元
        for col in cat_cols:
            if pd.api.types.is_categorical_dtype(df[col]):
                out[col] = out[col].astype(df[col].dtype)

    return out

In [36]:
hourly_filled = fill_missing_with_same_time_last_week(hourly_merged)
hourly_filled = hourly_filled.fillna(0)

/var/folders/8l/p7_mk6qs1y9dsdvmzt6qd68w0000gn/T/ipykernel_93752/1506172783.py:108: DeprecationWarning: is_categorical_dtype is deprecated and will be removed in a future version. Use isinstance(dtype, pd.CategoricalDtype) instead
  if pd.api.types.is_categorical_dtype(df[col]):


In [37]:
DataManager.show_status(hourly_filled)
print(len(hourly_filled))

+------------------------------+-----------+----------------+------------+----------+
|           Variable           |    Type   | Missing Values | Duplicates | Outliers |
+------------------------------+-----------+----------------+------------+----------+
|  A/C Set Temperature__A-25   | Numerical |       0        |   10784    |    0     |
|  A/C Set Temperature__A-26   | Numerical |       0        |   10813    |   246    |
| A/C Set Temperature__D-1南1  | Numerical |       0        |   10753    |    10    |
| A/C Set Temperature__D-2北1  | Numerical |       0        |   10758    |   311    |
| A/C Set Temperature__D-3南2  | Numerical |       0        |   10761    |    50    |
| A/C Set Temperature__D-4北2  | Numerical |       0        |   10754    |   226    |
| A/C Set Temperature__D-5南1  | Numerical |       0        |   10765    |    0     |
| A/C Set Temperature__D-6北1  | Numerical |       0        |   10761    |    0     |
| A/C Set Temperature__D-7南2  | Numerical |       0        |

In [ ]:
weather_df = pd.read_csv(WEATHER_CSV)
base_df = hourly_filled.copy().rename_axis("Datetime_hour").reset_index()
print(len(weather_df))
weather_df["Datetime_hour"] = pd.to_datetime(weather_df["datetime"])
base_df["Datetime_hour"] = pd.to_datetime(base_df["Datetime_hour"])
hourly_merged = base_df.merge(right=weather_df, on="Datetime_hour", how="inner")

10944


In [ ]:
# hourly_merged.to_csv("../data/base/hourly_filled.csv", index=False)

---

**予測モデル構築：ML-modeling**


In [56]:
base_df = pd.read_csv("../data/base/hourly_filled.csv")
base_df["Datetime_hour"] = pd.to_datetime(base_df["Datetime_hour"])
base_df = base_df.set_index("Datetime_hour", drop=True)

- 特徴量作成


In [ ]:
from __future__ import annotations
import pandas as pd
import numpy as np
from typing import Iterable, Optional, Dict, Any
from xgboost import XGBRegressor
from sklearn.multioutput import MultiOutputRegressor
from sklearn.metrics import r2_score, mean_absolute_error

# ========= 共通ユーティリティ =========


def _ensure_dtindex(df: pd.DataFrame) -> pd.DataFrame:
    if not isinstance(df.index, pd.DatetimeIndex):
        raise ValueError("DataFrame.index は DatetimeIndex 必須です。")
    out = df.sort_index()
    if out.index.has_duplicates:
        out = out[~out.index.duplicated(keep="last")]
    return out


def make_time_features(index: pd.DatetimeIndex) -> pd.DataFrame:
    s = index
    return pd.DataFrame(
        {
            "hour": s.hour,
            "month": s.month,
            "weekday": s.weekday,
            "is_weekend": (s.weekday >= 5).astype(int),
            # 周期特徴を使うなら↓を追加
            # "sin_hour": np.sin(2*np.pi*s.hour/24), "cos_hour": np.cos(2*np.pi*s.hour/24),
        },
        index=index,
    )


def compute_same_time_baseline(
    df: pd.DataFrame,
    cols: Iterable[str],
    *,
    days: int = 7,
    fallback: float = 0.0,  # 過去に観測ゼロの場合（データ冒頭など）の埋め
) -> pd.DataFrame:
    """
    直近N日分の「同一時刻」の平均（NaN無視）を列ごとに返す。
    未来情報リークなし。返り値の index/columns は df と同じ。
    """
    df = _ensure_dtindex(df)
    cols = list(cols)
    shifted_list = [
        df[cols].shift(freq=pd.Timedelta(days=k)) for k in range(1, days + 1)
    ]
    sum_vals, count_vals = None, None
    for s in shifted_list:
        sum_vals = s if sum_vals is None else sum_vals.add(s, fill_value=0.0)
        c = s.notna().astype(int)
        count_vals = c if count_vals is None else count_vals.add(c, fill_value=0)
    baseline = sum_vals.divide(count_vals).where(count_vals > 0, other=float(fallback))
    return baseline


def add_baseline_and_residual(
    df: pd.DataFrame,
    cols: Iterable[str],
    *,
    days: int = 7,
    fallback: float = 0.0,
    bl_prefix: str = "bl__",
    res_prefix: str = "res__",
    include_original: bool = True,
) -> pd.DataFrame:
    """
    指定列に対し baseline と residual (= original - baseline) を作り、列を増やして返す。
    """
    cols = list(cols)
    bl = compute_same_time_baseline(df, cols, days=days, fallback=fallback)
    res = df[cols] - bl
    out = pd.DataFrame(index=df.index)
    if include_original:
        out[cols] = df[cols]
    out[[f"{bl_prefix}{c}" for c in cols]] = bl
    out[[f"{res_prefix}{c}" for c in cols]] = res
    return out


# ========= 対象列の検出 =========


def pick_cols(df: pd.DataFrame, prefix: str) -> list[str]:
    return [c for c in df.columns if c.startswith(prefix)]


# ========= 特徴量（X）構築（残差入り） =========
def add_lagged_features(
    df: pd.DataFrame,
    cols: Iterable[str],
    *,
    lags_hours=(1, 2, 3, 24, 24 * 7),
    use_freq_shift=False,  # True: 時刻ベースでずらす / False: 行ベース
    prefix_fmt="lag{h}h__",
) -> pd.DataFrame:
    """指定列の過去ラグを作る（現在時刻は使わないのでリーク無し）。"""
    df = _ensure_dtindex(df)
    cols = list(cols)
    parts = []
    for h in lags_hours:
        lagged = (
            df[cols].shift(freq=pd.Timedelta(hours=h))
            if use_freq_shift
            else df[cols].shift(h)
        )
        lagged = lagged.add_prefix(prefix_fmt.format(h=h))
        parts.append(lagged)
    return pd.concat(parts, axis=1)


def build_features_residualized(
    base_df: pd.DataFrame,
    *,
    indoor_prefix="Indoor Temp.__",
    setT_prefix="A/C Set Temperature__",
    mode_prefix="A/C Mode__",
    wind_prefix="A/C Fan Speed__",
    onoff_prefix="A/C ON/OFF__",
    weather_cols: Optional[
        Iterable[str]
    ] = None,  # 例: ["Outdoor Temp.", "Outdoor Humidity", "Solar Radiation"]
    days_for_baseline: int = 7,
    baseline_fallback: float = 0.0,
    include_original_controls: bool = True,
    lags_hours=(1, 2, 3, 24, 24 * 7),  # ★ 室内温度/室外機kWhのラグ
    use_freq_shift=False,  # 欠番が多いとき True で時刻基準に
) -> pd.DataFrame:
    """
    X = [時間特徴]
        + [室温: baseline とラグ群]
        + [室外機kWh: baseline とラグ群]
        + [天気の原値/BL/RES（任意）]
        + [操作量（原値）]
    """
    df = _ensure_dtindex(base_df)
    parts = []

    # 時間特徴
    parts.append(make_time_features(df.index))

    # ---- 室内温度（indoor_temp__* に改名）: baseline + ラグ（t-1等）----
    indoor_cols_raw = pick_cols(df, indoor_prefix)
    if indoor_cols_raw:
        tmp = df[indoor_cols_raw].copy()
        renamed = {
            c: c.replace(indoor_prefix, "indoor_temp__") for c in indoor_cols_raw
        }
        tmp.rename(columns=renamed, inplace=True)

        # baseline（過去日の同時刻平均; 現在時刻で使ってOK）
        bl_in = compute_same_time_baseline(
            tmp, tmp.columns, days=days_for_baseline, fallback=baseline_fallback
        )
        parts.append(bl_in.add_prefix("bl__"))

        # ラグ（現在の生値は入れない）
        lag_in = add_lagged_features(
            tmp, tmp.columns, lags_hours=lags_hours, use_freq_shift=use_freq_shift
        )
        parts.append(lag_in)

    # ---- 室外機 kWh（total_kwh__*）: baseline + ラグ（t-1等）----
    odu_cols = pick_cols(df, "total_kwh__")
    if odu_cols:
        bl_odu = compute_same_time_baseline(
            df, odu_cols, days=days_for_baseline, fallback=baseline_fallback
        )
        parts.append(bl_odu.add_prefix("bl__"))

        lag_odu = add_lagged_features(
            df, odu_cols, lags_hours=lags_hours, use_freq_shift=use_freq_shift
        )
        parts.append(lag_odu)

    # ---- 天気 → 残差とBL（任意; 既存ロジックを継続）----
    if weather_cols is None:
        candidates = ["Outdoor Temp.", "Outdoor Humidity", "Solar Radiation"]
        weather_cols = [c for c in candidates if c in df.columns]
    weather_cols = list(weather_cols)
    if weather_cols:
        parts.append(
            add_baseline_and_residual(
                df,
                cols=weather_cols,
                days=days_for_baseline,
                fallback=baseline_fallback,
                bl_prefix="bl__",
                res_prefix="res__",
                include_original=True,
            )
        )

    # ---- 操作量（set/mode/wind/onoff）は原値のみ（時刻 t の操作は観測可能想定）----
    if include_original_controls:
        control_cols = []
        for p in (setT_prefix, mode_prefix, wind_prefix, onoff_prefix):
            control_cols += pick_cols(df, p)
        if control_cols:
            parts.append(df[control_cols].copy())

    X = pd.concat(parts, axis=1)
    X = X.apply(pd.to_numeric, errors="coerce").fillna(0.0)
    return X


# ========= 目的変数（Y）を残差化 =========


def build_targets_residualized(
    base_df: pd.DataFrame,
    *,
    odu_prefix="total_kwh__",
    indoor_prefix="Indoor Temp.__",
    days_for_baseline: int = 7,
    baseline_fallback: float = 0.0,
    out_indoor_prefix="indoor_temp__",
) -> pd.DataFrame:
    """
    返り値: Y_res（= 元値 - 同時刻過去平均BL）のみを DataFrame で返す。
    列: total_kwh__*, indoor_temp__*（出力名で統一）
    """
    df = _ensure_dtindex(base_df)

    # 出力列の収集
    y_odu_cols = pick_cols(df, odu_prefix)
    y_ind_cols_raw = pick_cols(df, indoor_prefix)
    y_ind_cols_map = {
        c: c.replace(indoor_prefix, out_indoor_prefix) for c in y_ind_cols_raw
    }

    # 元のターゲット（出力名で統一）
    Y_raw = pd.DataFrame(index=df.index)
    if y_odu_cols:
        Y_raw[y_odu_cols] = df[y_odu_cols]
    if y_ind_cols_raw:
        Y_raw[list(y_ind_cols_map.values())] = df[y_ind_cols_raw].rename(
            columns=y_ind_cols_map
        )

    # 同時刻過去平均（BL）→ 残差のみ返す
    Y_bl = compute_same_time_baseline(
        Y_raw, Y_raw.columns, days=days_for_baseline, fallback=baseline_fallback
    )
    Y_res = Y_raw - Y_bl
    return Y_res

In [ ]:
X_features = build_features_residualized(base_df)
Y_features = build_targets_residualized(base_df)

In [71]:
DataManager.show_status(X_features)

+-------------------------------+-----------+----------------+------------+----------+
|            Variable           |    Type   | Missing Values | Duplicates | Outliers |
+-------------------------------+-----------+----------------+------------+----------+
|              hour             | Numerical |       0        |   11051    |    0     |
|             month             | Numerical |       0        |   11062    |    0     |
|            weekday            | Numerical |       0        |   11068    |    0     |
|           is_weekend          | Numerical |       0        |   11073    |    0     |
|     bl__indoor_temp__A-25     | Numerical |       0        |    6389    |    24    |
|     bl__indoor_temp__A-26     | Numerical |       0        |    6363    |    24    |
|    bl__indoor_temp__D-1南1    | Numerical |       0        |    6951    |   150    |
|    bl__indoor_temp__D-2北1    | Numerical |       0        |    6266    |    87    |
|    bl__indoor_temp__D-3南2    | Numerical | 

- モデル構築


In [65]:
def _get_Y_res(Y_features) -> pd.DataFrame:
    """Y_features が dict なら Y_res を、DataFrame ならそれを残差として返す。"""
    if isinstance(Y_features, dict):
        return Y_features["Y_res"]
    if isinstance(Y_features, pd.DataFrame):
        return Y_features
    raise TypeError(
        "Y_features は dict（Y_res を含む）か DataFrame（残差のみ）を渡してください。"
    )


def _timewise_split(
    X: pd.DataFrame,
    Y_res: pd.DataFrame,
    train_ratio: float = 0.8,
) -> Dict[str, Any]:
    """時系列順に分割（Y残差で欠損行を落としてから分割）。"""
    XY = X.join(Y_res, how="inner").dropna(subset=Y_res.columns.tolist())
    n = len(XY)
    n_tr = int(n * train_ratio)
    X_cols, Y_cols = list(X.columns), list(Y_res.columns)
    return {
        "XY": XY,
        "X_cols": X_cols,
        "Y_cols": Y_cols,
        "X_tr": XY[X_cols].values[:n_tr],
        "X_te": XY[X_cols].values[n_tr:],
        "Y_tr": XY[Y_cols].values[:n_tr],
        "Y_te": XY[Y_cols].values[n_tr:],
        "idx_tr": XY.index[:n_tr],
        "idx_te": XY.index[n_tr:],
    }


def train_xgb_multioutput(X_tr: np.ndarray, Y_tr: np.ndarray, **xgb_kwargs):
    """MultiOutputRegressor(XGBRegressor) で残差を学習。"""
    params = dict(
        n_estimators=600,
        learning_rate=0.05,
        max_depth=6,
        subsample=0.8,
        colsample_bytree=0.8,
        tree_method="hist",
        n_jobs=-1,
        random_state=42,
    )
    params.update(xgb_kwargs)
    model = MultiOutputRegressor(XGBRegressor(**params))
    model.fit(X_tr, Y_tr)
    return model


def evaluate_predictions(
    Y_true: np.ndarray, Y_pred: np.ndarray, target_names: list[str]
) -> pd.DataFrame:
    r2 = [r2_score(Y_true[:, i], Y_pred[:, i]) for i in range(Y_true.shape[1])]
    mae = [
        mean_absolute_error(Y_true[:, i], Y_pred[:, i]) for i in range(Y_true.shape[1])
    ]
    return pd.DataFrame({"target": target_names, "R2": r2, "MAE": mae}).set_index(
        "target"
    )


def fit_predict_eval_xgb_residual_only(
    X_features: pd.DataFrame,
    Y_features,  # dict(Y_res=...) または DataFrame(残差のみ)
    *,
    train_ratio: float = 0.8,
    xgb_params: Optional[Dict[str, Any]] = None,
):
    """
    残差のみを予測して評価（R2/MAE）を返すワンストップ関数。
    """
    xgb_params = xgb_params or {}

    # Y は残差のみ
    Y_res = _get_Y_res(Y_features)

    # 分割
    pack = _timewise_split(X_features, Y_res, train_ratio=train_ratio)

    # 学習
    model = train_xgb_multioutput(pack["X_tr"], pack["Y_tr"], **xgb_params)

    # 予測（残差）
    Y_res_hat_tr = model.predict(pack["X_tr"])
    Y_res_hat_te = model.predict(pack["X_te"])

    # 評価（残差空間）
    metrics_tr = evaluate_predictions(pack["Y_tr"], Y_res_hat_tr, pack["Y_cols"])
    metrics_te = evaluate_predictions(pack["Y_te"], Y_res_hat_te, pack["Y_cols"])

    return {
        "model": model,
        "X_cols": pack["X_cols"],
        "Y_cols": pack["Y_cols"],
        "idx_tr": pack["idx_tr"],
        "idx_te": pack["idx_te"],
        "Y_res_hat_tr": Y_res_hat_tr,
        "Y_res_hat_te": Y_res_hat_te,
        "metrics_tr_res": metrics_tr,
        "metrics_te_res": metrics_te,
    }

In [66]:
res = fit_predict_eval_xgb_residual_only(
    X_features,
    Y_features,
    train_ratio=0.8,
    xgb_params=dict(n_estimators=800, learning_rate=0.05),
)

In [69]:
print(res["metrics_te_res"].sort_values("R2"))

                           R2          MAE
target                                    
total_kwh__49-9     -0.028363  1602.294809
total_kwh__44-5      0.294733  5939.981118
total_kwh__44-6      0.372372  1575.055114
total_kwh__49-1      0.381582  5074.254839
total_kwh__49-7      0.437781  1463.698779
total_kwh__44-3      0.471412  2538.668696
total_kwh__44-2      0.478233  1786.641481
total_kwh__44-4      0.479936   588.064108
total_kwh__49-6      0.503536  2880.428366
total_kwh__49-8      0.511001  1153.164057
total_kwh__49-4      0.523257  2244.828454
total_kwh__43-3      0.534791   298.933555
indoor_temp__D-1南1   0.537879     1.042671
total_kwh__44-1      0.557353  3688.904823
total_kwh__49-2      0.576822  3534.221836
total_kwh__44-8      0.591774   850.279493
indoor_temp__D-2北1   0.597740     1.405931
total_kwh__49-3      0.617505  1842.216357
total_kwh__41-1      0.624448  9202.656392
total_kwh__44-7      0.670711  2901.200285
total_kwh__43-2      0.675258  6934.232564
indoor_temp

- 元のスケールの戻すコード


In [70]:
def _make_target_frame_from_base(
    base_df: pd.DataFrame,
    y_cols: list[str],
    *,
    indoor_prefix="Indoor Temp.__",
    out_indoor_prefix="indoor_temp__",
) -> pd.DataFrame:
    """base_dfから y_cols（total_kwh__*, indoor_temp__*）に対応する元系列を集めて返す。"""
    df = pd.DataFrame(index=base_df.index)
    for c in y_cols:
        if c.startswith("total_kwh__"):
            if c in base_df.columns:
                df[c] = base_df[c]
        elif c.startswith(out_indoor_prefix):
            orig = indoor_prefix + c[len(out_indoor_prefix) :]
            if orig in base_df.columns:
                df[c] = base_df[orig].values
    return df


def restore_raw_predictions(
    base_df: pd.DataFrame,
    res_pack: dict,  # fit_predict_eval_xgb_residual_only の戻り値
    *,
    split: str = "te",  # "tr" or "te"（どちらを復元するか）
    days_for_baseline: int = 7,
    baseline_fallback: float = 0.0,
    indoor_prefix: str = "Indoor Temp.__",
    out_indoor_prefix: str = "indoor_temp__",
    horizon_hours: int = 0,  # 先読み学習している場合はそのホライズン（例: 1）
    use_freq_shift: bool = True,  # 欠番があるなら True 推奨
) -> pd.DataFrame:
    """
    残差予測を Raw に復元して DataFrame を返す。
    baselineは base_df から再計算して、必要なら t+Δ にシフトする。
    """
    y_cols = res_pack["Y_cols"]
    idx = res_pack["idx_tr"] if split == "tr" else res_pack["idx_te"]
    y_res_hat = res_pack["Y_res_hat_tr"] if split == "tr" else res_pack["Y_res_hat_te"]

    # 1) 対象列の元系列を base_df から作る（列名は y_cols と揃える）
    Y_src = _make_target_frame_from_base(
        base_df,
        y_cols,
        indoor_prefix=indoor_prefix,
        out_indoor_prefix=out_indoor_prefix,
    )

    # 2) baseline 再計算（過去N日同時刻平均）
    Y_bl = compute_same_time_baseline(
        Y_src, y_cols, days=days_for_baseline, fallback=baseline_fallback
    )

    # 3) 先読み学習（t→t+Δ）をしているなら baseline を +Δ へずらす
    if horizon_hours != 0:
        if use_freq_shift:
            Y_bl = Y_bl.shift(freq=pd.Timedelta(hours=horizon_hours))
        else:
            Y_bl = Y_bl.shift(-horizon_hours)

    # 4) インデックスを合わせて復元
    Y_bl_idx = Y_bl.loc[idx].values
    Y_raw_hat = Y_bl_idx + y_res_hat

    return pd.DataFrame(Y_raw_hat, index=idx, columns=y_cols)

In [74]:
y_hat_raw_te = restore_raw_predictions(base_df, res, split="te", horizon_hours=0)

In [75]:
y_hat_raw_te

,total_kwh__41-1,total_kwh__43-1,total_kwh__43-2,total_kwh__43-3,total_kwh__43-4,total_kwh__44-1,total_kwh__44-2,total_kwh__44-3,total_kwh__44-4,total_kwh__44-5,...,indoor_temp__E-16北4,indoor_temp__E-17,indoor_temp__E-9南1,indoor_temp__F-18,indoor_temp__F-19,indoor_temp__F-20,indoor_temp__G-21,indoor_temp__G-22,indoor_temp__G-23,indoor_temp__G-24
Datetime_hour,,,,,,,,,,,,,,,,,,,,,
2025-06-29 10:00:00,1124.958984,4142.342076,1464.109096,342.479444,-99.416713,720.725725,332.550854,773.150497,266.458890,12800.274833,...,36.847046,30.930742,35.854102,30.950107,30.640197,31.784449,36.972454,30.494304,38.325301,37.086250
2025-06-29 11:00:00,1459.870135,2816.935268,2830.525670,373.600237,-586.584821,615.255702,573.987943,713.266666,287.005401,13700.138742,...,33.596811,29.868709,35.153955,30.232172,30.427216,30.535781,34.280748,29.980717,35.184769,33.777426
2025-06-29 12:00:00,204.873553,1810.567801,1535.204520,394.200842,775.306920,934.494594,601.522716,974.522003,240.144517,13227.163156,...,31.561583,29.665099,33.220141,29.814593,29.838320,30.068011,32.842727,29.671074,33.264571,32.297891
2025-06-29 13:00:00,-70.787303,2835.679688,1342.350586,329.621620,1487.905692,962.822348,521.889121,743.488886,354.911308,9364.068499,...,30.157746,29.661177,29.984569,29.823030,29.614717,30.403362,31.976655,28.873929,31.686315,30.511158
2025-06-29 14:00:00,265.862904,143.029157,1353.312221,334.506207,4.822266,859.070452,401.129728,713.294879,265.622792,4766.884138,...,28.886574,29.812977,28.769674,29.599171,29.681645,30.220420,28.887828,28.213645,30.379620,29.529725
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2025-09-27 21:00:00,241.519073,13047.724679,3047.342634,399.656281,2327.382987,2202.850621,764.981814,3295.764805,306.383584,22110.183105,...,18.720092,23.943050,19.583443,26.893631,26.005273,27.486987,21.688467,21.731035,21.299187,20.679185
2025-09-27 22:00:00,-27.615860,11989.517543,5324.987653,383.430049,5505.413783,1085.885254,613.218498,6338.150112,463.364495,22600.609828,...,19.358614,23.688891,20.018958,26.883667,26.409290,28.300152,22.261525,22.163872,22.633746,21.552330
2025-09-27 23:00:00,1528.620143,13832.017508,3151.781669,632.745172,3021.859933,1483.892160,630.902762,3331.459874,343.404489,23762.073730,...,20.799086,24.412348,21.831480,27.065158,27.645404,29.970846,23.737970,23.422343,25.657635,23.956766
